In [2]:
import requests

In [ ]:
access_token = ''
ids= ['gabgalani',]
fields = [
  '{media{comments_count,like_count, media_type, media_url, caption, timestamp, permalink, media_product_type,thumbnail_url, children{media_type, media_url}, owner{id, username}}}'
]
instagram_name = 'raentretenimento'

In [ ]:
def get_others_accounts_info(access_token, ids, fields, output_path, period, instagram_name):
    all_data = []

    for username, id_ig in ids:
        url = f"https://graph.facebook.com/v19.0/{id_ig}"
        params = {
            "fields": f"business_discovery.username({instagram_name}){','.join(fields)}",
            "access_token": access_token
        }
        
        if period:
            params["period"] = period
        
        base_url = url
        page = 0
        while base_url:
            page = page + 1
            response = requests.get(base_url, params=params)
            data = response.json()

            # Verifique se a resposta contém erros
            if 'error' in data:
                print(f"Error fetching data for {username}: {data['error']['message']}")
                break
            
            dic = ''
            if 'business_discovery' in data:
                if 'data' in data:
                    # Caso onde 'data' está na raiz do JSON
                    extracted_data = data['data']
                    aux = False

                elif 'media' in data['business_discovery']: #and 'data' in data['media']:
                    extracted_data = data['business_discovery']['media']['data']
                    print('--------------------------------- PASSEI AQUI -------------------------------------')
                    dic = 'media'
                    aux = False
                
                elif 'insights' in data:
                    extracted_data = data['insights']['data']
                    dic = 'insights'
                    aux = False

                elif 'stories' in data:
                    extracted_data = data['stories']['data']
                    dic = 'stories'
                    aux = False

                else:                
                    print(f"--------------- Unexpected data format for {username}  ------------------")
                    extracted_data = (data)
                    aux = True
                    print(output_path)
                    pass
            
            # Adicione o username e os dados extraídos à lista de todos os dados
            if aux == False:
                for item in extracted_data:
                    item['username'] = username
                    all_data.append(item)
            else:
                all_data.append(extracted_data)


            after = data['business_discovery'].get(dic, {}).get('paging', {}).get('cursors', {}).get('after')
            print(after)

            if page == 2: 
                # print(data)
                after = data['business_discovery'].get(dic, {}).get('paging', {}).get('cursors', {}).get('after')
                print(data)
                sys.exit()

            print(f'passei a pagina: {page} ')

            # Após a primeira solicitação, remova os parâmetros para evitar duplicidade
            params = {
                "fields": f"business_discovery.username({instagram_name}){','.join(fields)}",
                "access_token": access_token,
                "after": after
            }

    # Escreva todos os dados em um arquivo
    with open(output_path, 'w') as json_file:
        json.dump(all_data, json_file, indent=4)